In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
lines = spark.read.text('file:///home/talentum/test-jupyter/P2/M2/SM4/constitution.txt') 

In [4]:
lines.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
lines.show(truncate=False)

+-------------------------------------------------------------------------+
|value                                                                    |
+-------------------------------------------------------------------------+
|We the People of the United States, in Order to form a more perfect      |
|Union, establish Justice, insure domestic Tranquility, provide for the   |
|common defence, promote the general Welfare, and secure the Blessings of |
|Liberty to ourselves and our Posterity, do ordain and establish this     |
|Constitution for the United States of America.                           |
|Article 1.  Section 1 All legislative Powers herein granted shall be     |
|vested in a Congress of the United States, which shall consist of a      |
|Senate and House of Representatives.  Section 2 The House of             |
|Representatives shall be composed of Members chosen every second Year by |
|the People of the several States, and the Electors in each State shall   |
|have the Qu

In [6]:
import pyspark.sql.functions as F

lines = lines.withColumn('splits', F.split(lines.value, ' '))

In [7]:
lines.show(truncate=False)

+-------------------------------------------------------------------------+--------------------------------------------------------------------------------------+
|value                                                                    |splits                                                                                |
+-------------------------------------------------------------------------+--------------------------------------------------------------------------------------+
|We the People of the United States, in Order to form a more perfect      |[We, the, People, of, the, United, States,, in, Order, to, form, a, more, perfect, ]  |
|Union, establish Justice, insure domestic Tranquility, provide for the   |[Union,, establish, Justice,, insure, domestic, Tranquility,, provide, for, the, ]    |
|common defence, promote the general Welfare, and secure the Blessings of |[common, defence,, promote, the, general, Welfare,, and, secure, the, Blessings, of, ]|
|Liberty to ourselves 

In [8]:
lines.printSchema()

root
 |-- value: string (nullable = true)
 |-- splits: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [9]:
lines.select('splits').show(truncate=False)

+--------------------------------------------------------------------------------------+
|splits                                                                                |
+--------------------------------------------------------------------------------------+
|[We, the, People, of, the, United, States,, in, Order, to, form, a, more, perfect, ]  |
|[Union,, establish, Justice,, insure, domestic, Tranquility,, provide, for, the, ]    |
|[common, defence,, promote, the, general, Welfare,, and, secure, the, Blessings, of, ]|
|[Liberty, to, ourselves, and, our, Posterity,, do, ordain, and, establish, this, ]    |
|[Constitution, for, the, United, States, of, America., , ]                            |
|[Article, 1., , Section, 1, All, legislative, Powers, herein, granted, shall, be, ]   |
|[vested, in, a, Congress, of, the, United, States,, which, shall, consist, of, a, ]   |
|[Senate, and, House, of, Representatives., , Section, 2, The, House, of, ]            |
|[Representatives, sh

In [10]:
lines = lines.select(F.explode(lines.splits).alias("word")) # Basically word tokenize in form of DataFrame

In [11]:
lines.printSchema()

root
 |-- word: string (nullable = true)



In [12]:
wordcount = lines.groupBy('word').count()

In [13]:
wordcount.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = false)



In [15]:
wordcount.orderBy('count',ascending=False).show(100)

+---------------+-----+
|           word|count|
+---------------+-----+
|               |  812|
|            the|  662|
|             of|  493|
|          shall|  293|
|            and|  256|
|             to|  183|
|             be|  178|
|             or|  157|
|             in|  137|
|             by|  100|
|              a|   94|
|         United|   85|
|            for|   81|
|            any|   79|
|      President|   72|
|            The|   64|
|             as|   64|
|           have|   63|
|        States,|   55|
|           such|   52|
|          State|   47|
|         States|   46|
|            may|   42|
|          which|   42|
|            not|   41|
|           from|   40|
|       Congress|   39|
|             on|   39|
|            all|   37|
|           this|   30|
|          other|   30|
|      Amendment|   28|
|          their|   28|
|          House|   27|
|     President,|   27|
|             he|   25|
|           Vice|   24|
|            one|   24|
|             at

In [17]:
import pandas as pd

ValueError: DataFrame constructor not properly called!